# Final data

In [2]:
from plots import before_after_plot, plot_mean_welch
from get_bads import get_bad_epochs, get_bad_channels
from preprocess import preprocess
import numpy as np

## Splitting per participant

In [9]:
bands = [(0.5,4), (4,9), (9,14), (14,30), (30,49)]
band_names = ('delta','theta','alpha','beta','gamma')

participants = ['k'+str(i) for i in range(1,19)]
participants.extend(['m'+str(i) for i in range(1,19)])
participants.pop(0)
participants.pop(0)
participants.pop(14)
package = np.empty((5,0,19,77))
epo_class = np.empty((0))
np.random.shuffle(participants)
ind_stop = (7,12,17,22,27,32)
# ind_stop = (2,5,8,11,14,17)

np.save('participants.npy', participants)    

for i in range(len(participants)):
    fs, epochs, channels, y = preprocess('./data2/'+participants[i]+'.set', 
                        fmin = 0.5, fmax = 49, 
                        method = 'iir', 
                        down_fs = 256, 
                        tmin = -0.2, tmax = 1, 
                        exclude_ratio = 2, 
                        method_bands = 'iir', 
                        bands = bands,
                        verbose = False,
                        notch = True)
    package = np.concatenate((package, epochs), axis = 1)
    epo_class = np.concatenate((epo_class, y))
    print("DONE, PARTICIPANT: " + participants[i])
    if i in ind_stop:
        np.save('data_no'+str(i+1)+'.npy', package)
        np.save('class_no'+str(i+1)+'.npy', epo_class)
        package = np.empty((5,0,19,77))
        epo_class = np.empty((0))
        print("SAVED PACKAGE, NUMBER: " + str(i))
np.save('channels.npy', np.array(channels))    
print("FINISHED")

DONE, PARTICIPANT: k8
DONE, PARTICIPANT: k10
DONE, PARTICIPANT: k16
SAVED PACKAGE, NUMBER: 2
DONE, PARTICIPANT: k3
DONE, PARTICIPANT: k12
DONE, PARTICIPANT: k6
SAVED PACKAGE, NUMBER: 5
DONE, PARTICIPANT: k13
DONE, PARTICIPANT: k5
DONE, PARTICIPANT: k18
SAVED PACKAGE, NUMBER: 8
DONE, PARTICIPANT: k11
DONE, PARTICIPANT: k15
DONE, PARTICIPANT: k7
SAVED PACKAGE, NUMBER: 11
DONE, PARTICIPANT: k4
DONE, PARTICIPANT: k14
DONE, PARTICIPANT: k9
SAVED PACKAGE, NUMBER: 14
FINISHED


## Mixing participants

In [101]:
bands = [(0.5,4), (4,9), (9,14), (14,30), (30,49)]
band_names = ('delta','theta','alpha','beta','gamma')

participants = ['k'+str(i) for i in range(1,19)]
participants.extend(['m'+str(i) for i in range(1,19)])
participants.pop(0)
participants.pop(0)
participants.pop(14)

packages = {1 : np.empty((5,0,19,77)), 2: np.empty((5,0,19,77)), 3: np.empty((5,0,19,77)), 4: np.empty((5,0,19,77)), 5: np.empty((5,0,19,77))}
epo_class = {1 : np.empty((0)), 2: np.empty((0)), 3: np.empty((0)), 4: np.empty((0)), 5: np.empty((0))}
package_0k = np.empty((5,0,19,77))
epo_class_0k = np.empty((0))

for participant in participants:
    print(participant)
    # epochs shape = (freq bands:5, n.o. epochs, n.o. channels: 19, n.o. samples: 77)
    fs, epochs, channels, y = preprocess('./data2/'+participant+'.set', 
                        fmin = 0.5, fmax = 49, 
                        method = 'iir', 
                        down_fs = 256, 
                        tmin = -0.2, tmax = 1, 
                        exclude_ratio = 2, 
                        method_bands = 'iir', 
                        bands = bands,
                        verbose = False,
                        notch = True)
    test_count = int(epochs.shape[1]*0.25)
    package_0k = np.concatenate((package_0k, epochs[:,0:test_count,:,:]), axis = 1)
    epo_class_0k = np.concatenate((epo_class_0k, y[0:test_count]))
    train_val_count = epochs.shape[1]-int(epochs.shape[1]*0.25)
    for i in range(0,5):
        q1, q2 = i*train_val_count//5+test_count, (i+1)*train_val_count//5+test_count
        temp = np.concatenate((packages[i+1], epochs[:,q1:q2,:,:]), axis = 1)
        packages[i+1] = temp
        y_temp = np.concatenate((epo_class[i+1], y[q1:q2]))
        epo_class[i+1] = y_temp
    print("DONE, PARTICIPANT: " + participant)
print("FINISHED")

k3
DONE, PARTICIPANT: k3
k4
DONE, PARTICIPANT: k4
k5
DONE, PARTICIPANT: k5
k6
DONE, PARTICIPANT: k6
k7
DONE, PARTICIPANT: k7
k8
DONE, PARTICIPANT: k8
k9
DONE, PARTICIPANT: k9
k10
DONE, PARTICIPANT: k10
k11
DONE, PARTICIPANT: k11
k12
DONE, PARTICIPANT: k12
k13
DONE, PARTICIPANT: k13
k14
DONE, PARTICIPANT: k14
k15
DONE, PARTICIPANT: k15
k16
DONE, PARTICIPANT: k16
k18
DONE, PARTICIPANT: k18
m1
DONE, PARTICIPANT: m1
m2
DONE, PARTICIPANT: m2
m3
DONE, PARTICIPANT: m3
m4
DONE, PARTICIPANT: m4
m5
DONE, PARTICIPANT: m5
m6
DONE, PARTICIPANT: m6
m7
DONE, PARTICIPANT: m7
m8
DONE, PARTICIPANT: m8
m9
DONE, PARTICIPANT: m9
m10
DONE, PARTICIPANT: m10
m11
DONE, PARTICIPANT: m11
m12
DONE, PARTICIPANT: m12
m13
DONE, PARTICIPANT: m13
m14
DONE, PARTICIPANT: m14
m15
DONE, PARTICIPANT: m15
m16
DONE, PARTICIPANT: m16
m17
DONE, PARTICIPANT: m17
m18
DONE, PARTICIPANT: m18
FINISHED


In [109]:
for i in range(1,6):
    np.save('npy_data_mix/data_no'+str(i)+'.npy', packages[i])
    np.save('npy_data_mix/class_no'+str(i)+'.npy', epo_class[i])

In [110]:
np.save('npy_data_mix/data_no0.npy', package_0k)
np.save('npy_data_mix/class_no0.npy', epo_class_0k)

## All participants separately

In [1]:
bands = [(0.5,4), (4,9), (9,14), (14,30), (30,49)]
band_names = ('delta','theta','alpha','beta','gamma')

participants = ['k'+str(i) for i in range(1,19)]
participants.extend(['m'+str(i) for i in range(1,19)])
participants.pop(0)
participants.pop(0)
participants.pop(14)

for i, participant in enumerate(participants):
    fs, epochs, channels, y = preprocess('./data2/'+participant+'.set', 
                        fmin = 0.5, fmax = 49, 
                        method = 'iir', 
                        down_fs = 256, 
                        tmin = -0.2, tmax = 1, 
                        exclude_ratio = 2, 
                        method_bands = 'iir', 
                        bands = bands,
                        verbose = False,
                        notch = True)
    np.save(f'npy_data_all_separate/data_part{i+1}.npy', epochs)
    np.save(f'npy_data_all_separate/class_part{i+1}.npy', epochs)
    print("DONE, PARTICIPANT: " + participant)
print("FINISHED")

NameError: name 'preprocess' is not defined

# Get names of all interpolated channels for all participants

In [2]:
from preprocess import read_and_get_bad_channels

participants = ['k'+str(i) for i in range(1,19)]
participants.extend(['m'+str(i) for i in range(1,19)])
participants.pop(0)
participants.pop(0)
participants.pop(14)
dic_channels = {}
for i, participant in enumerate(participants):
    ch_ind, ch_names = read_and_get_bad_channels('./data2/'+participant+'.set', 
                        fmin = 0.5, fmax = 49, 
                        method = 'iir', notch = True)
    dic_channels[participant+"_names"] = ch_names
    dic_channels[participant+"_ind"] = ch_ind
    print(f"Participant: {participant}, ch names: {ch_names}")

Participant: k3, ch names: ['F8' 'Fp1' 'Fp2']
Participant: k4, ch names: ['Fp1' 'Fp2']
Participant: k5, ch names: ['Fp1']
Participant: k6, ch names: ['T3' 'T4' 'F8' 'Fp1' 'Fp2']
Participant: k7, ch names: ['F7']
Participant: k8, ch names: []
Participant: k9, ch names: ['T3' 'T4' 'Fp1' 'Fp2']
Participant: k10, ch names: ['Fp1' 'Fp2']
Participant: k11, ch names: ['Fp1' 'Fp2']
Participant: k12, ch names: ['Fp1' 'Fp2']
Participant: k13, ch names: ['O2' 'T5' 'Pz']
Participant: k14, ch names: ['T4' 'Fp1' 'Fp2']
Participant: k15, ch names: ['O2' 'T5' 'Fp1' 'Fp2']
Participant: k16, ch names: ['Fp1' 'Fp2']
Participant: k18, ch names: ['T5' 'Fp1' 'Fp2']
Participant: m1, ch names: ['T3' 'T4' 'Fp1' 'Fp2']
Participant: m2, ch names: ['T3' 'T4' 'Fp1' 'Fp2']
Participant: m3, ch names: ['Pz' 'C3' 'Fp1' 'Fp2']
Participant: m4, ch names: ['Fp1' 'Fp2']
Participant: m5, ch names: ['Fp1' 'Fp2']
Participant: m6, ch names: ['P3' 'Fp1' 'Fp2']
Participant: m7, ch names: ['O1' 'Fp1' 'Fp2']
Participant: m8, ch n

In [26]:
for key in dic_channels:
    dic_channels[key]=list(dic_channels[key])

In [27]:
print(dic_channels)

{'k3_names': ['Fp1', 'Fp2', 'F8'], 'k3_ind': [16, 17, 18], 'k4_names': ['Fp1', 'Fp2'], 'k4_ind': [17, 18], 'k5_names': ['Fp1'], 'k5_ind': [17], 'k6_names': ['F8', 'T4', 'Fp1', 'T3', 'Fp2'], 'k6_ind': [7, 11, 16, 17, 18], 'k7_names': ['F7'], 'k7_ind': [12], 'k8_names': [], 'k8_ind': [], 'k9_names': ['T4', 'T3', 'Fp1', 'Fp2'], 'k9_ind': [17, 18, 11, 7], 'k10_names': ['Fp1', 'Fp2'], 'k10_ind': [17, 18], 'k11_names': ['Fp1', 'Fp2'], 'k11_ind': [17, 18], 'k12_names': ['Fp1', 'Fp2'], 'k12_ind': [17, 18], 'k13_names': ['O2', 'T5', 'Pz'], 'k13_ind': [1, 2, 4], 'k14_names': ['T4', 'Fp1', 'Fp2'], 'k14_ind': [17, 18, 11], 'k15_names': ['O2', 'Fp1', 'Fp2', 'T5'], 'k15_ind': [1, 2, 18, 17], 'k16_names': ['Fp1', 'Fp2'], 'k16_ind': [17, 18], 'k18_names': ['Fp1', 'Fp2', 'T5'], 'k18_ind': [17, 2, 18], 'm1_names': ['T4', 'T3', 'Fp1', 'Fp2'], 'm1_ind': [17, 18, 11, 7], 'm2_names': ['T4', 'T3', 'Fp1', 'Fp2'], 'm2_ind': [17, 18, 11, 7], 'm3_names': ['Fp1', 'Fp2', 'C3', 'Pz'], 'm3_ind': [8, 17, 18, 4], 'm4_

In [28]:
import pickle

In [29]:
a_file = open("channels_interpolated.pkl", "wb")

pickle.dump(dic_channels, a_file)

a_file.close()


a_file = open("channels_interpolated.pkl", "rb")

output = pickle.load(a_file)

print(output)

{'k3_names': ['Fp1', 'Fp2', 'F8'], 'k3_ind': [16, 17, 18], 'k4_names': ['Fp1', 'Fp2'], 'k4_ind': [17, 18], 'k5_names': ['Fp1'], 'k5_ind': [17], 'k6_names': ['F8', 'T4', 'Fp1', 'T3', 'Fp2'], 'k6_ind': [7, 11, 16, 17, 18], 'k7_names': ['F7'], 'k7_ind': [12], 'k8_names': [], 'k8_ind': [], 'k9_names': ['T4', 'T3', 'Fp1', 'Fp2'], 'k9_ind': [17, 18, 11, 7], 'k10_names': ['Fp1', 'Fp2'], 'k10_ind': [17, 18], 'k11_names': ['Fp1', 'Fp2'], 'k11_ind': [17, 18], 'k12_names': ['Fp1', 'Fp2'], 'k12_ind': [17, 18], 'k13_names': ['O2', 'T5', 'Pz'], 'k13_ind': [1, 2, 4], 'k14_names': ['T4', 'Fp1', 'Fp2'], 'k14_ind': [17, 18, 11], 'k15_names': ['O2', 'Fp1', 'Fp2', 'T5'], 'k15_ind': [1, 2, 18, 17], 'k16_names': ['Fp1', 'Fp2'], 'k16_ind': [17, 18], 'k18_names': ['Fp1', 'Fp2', 'T5'], 'k18_ind': [17, 2, 18], 'm1_names': ['T4', 'T3', 'Fp1', 'Fp2'], 'm1_ind': [17, 18, 11, 7], 'm2_names': ['T4', 'T3', 'Fp1', 'Fp2'], 'm2_ind': [17, 18, 11, 7], 'm3_names': ['Fp1', 'Fp2', 'C3', 'Pz'], 'm3_ind': [8, 17, 18, 4], 'm4_

In [41]:
import numpy as np
lista = list(output.values())[::2]

In [43]:
from collections import Counter


c = Counter(x for xs in lista for x in set(xs))

print(c)

Counter({'Fp1': 28, 'Fp2': 27, 'T4': 6, 'T5': 5, 'F8': 4, 'T3': 4, 'F7': 3, 'O2': 3, 'Pz': 3, 'C3': 1, 'P3': 1, 'O1': 1})
